In [1]:
import os
import json
import numpy as np
from collections import defaultdict
import pprint
from nltk.translate.bleu_score import sentence_bleu

import matplotlib.pyplot as plt
%matplotlib inline

## Results Data

In [2]:
mod20_dout = '/root/data/home/hoyeung/blob_alfred_data/exp_all/model:seq2seq_per_subgoal,name:v2_epoch_50_baseline'
mod21_dout = '/root/data/home/hoyeung/blob_alfred_data/exp_all/model:seq2seq_per_subgoal,name:v2_epoch_50_enc_max_pool_aux_loss'
mod22_dout = '/root/data/home/hoyeung/blob_alfred_data/exp_all/model:seq2seq_per_subgoal,name:v2_epoch_43_obj_instance_enc_max_pool_aux_loss'
mod23_dout = '/root/data/home/hoyeung/blob_alfred_data/exp_all/model:seq2seq_per_subgoal,name:v2_epoch_40_obj_instance_enc_biattn_axu_loss'

In [3]:
# select by best BLEU score

seen_20_epoch = 11
seen_21_epoch = 11
seen_22_epoch = 11
seen_23_epoch = 25

unseen_20_epoch = 5
unseen_21_epoch = 6
unseen_22_epoch = 12
unseen_23_epoch = 25

In [4]:
def load_res(dout, epoch_num, split):
    path = os.path.join(dout, '{}.debug_epoch_{}.preds.json'.format(split, epoch_num))
    with open(path, 'r') as f:
        res = json.load(f)
        
    return res

In [5]:
mod20_train_sanity_res = load_res(mod20_dout, seen_20_epoch, 'train_sanity')
mod20_valid_seen_res = load_res(mod20_dout, seen_20_epoch, 'valid_seen')
mod20_valid_unseen_res = load_res(mod20_dout, unseen_20_epoch, 'valid_unseen')

mod21_train_sanity_res = load_res(mod21_dout, seen_21_epoch, 'train_sanity')
mod21_valid_seen_res = load_res(mod21_dout, seen_21_epoch, 'valid_seen')
mod21_valid_unseen_res = load_res(mod21_dout, unseen_21_epoch, 'valid_unseen')

mod22_train_sanity_res = load_res(mod22_dout, seen_22_epoch, 'train_sanity')
mod22_valid_seen_res = load_res(mod22_dout, seen_22_epoch, 'valid_seen')
mod22_valid_unseen_res = load_res(mod22_dout, unseen_22_epoch, 'valid_unseen')

mod23_train_sanity_res = load_res(mod23_dout, seen_23_epoch, 'train_sanity')
mod23_valid_seen_res = load_res(mod23_dout, seen_23_epoch, 'valid_seen')
mod23_valid_unseen_res = load_res(mod23_dout, unseen_23_epoch, 'valid_unseen')

In [6]:
chosen_train_sanity_task_ids = [
'trial_T20190908_115507_503798_0',
'trial_T20190907_011351_027508_0',
'trial_T20190910_055356_172564_0',
'trial_T20190919_045956_375063_0',
'trial_T20190918_173742_042769_0',
'trial_T20190908_030909_087593_0',
'trial_T20190909_004008_377860_0',
'trial_T20190908_072254_824822_0',
'trial_T20190907_181954_161870_0',
'trial_T20190909_113100_297495_0',
]

chosen_valid_seen_task_ids = [
'trial_T20190909_150916_682139_0', 
'trial_T20190908_192636_561572_0', 
'trial_T20190907_013001_399963_0', 
'trial_T20190907_074712_870488_0', 
'trial_T20190908_102840_789300_0', 
'trial_T20190909_031040_889814_0', 
'trial_T20190908_140701_251653_0', 
'trial_T20190909_055649_717880_0', 
'trial_T20190907_151802_277016_0', 
'trial_T20190909_032318_169393_0'
]

chosen_valid_unseen_task_ids = [
'trial_T20190909_123918_983775_0',
'trial_T20190909_123732_489327_0',
'trial_T20190906_191445_723170_0',
'trial_T20190907_020329_050110_0',
'trial_T20190907_170955_824797_0',
'trial_T20190909_210238_431966_0',
'trial_T20190906_224843_443882_0',
'trial_T20190906_214631_761426_0',
'trial_T20190907_074524_006355_0',
'trial_T20190908_114656_768805_0'
]

In [7]:
urls = {
    'trial_T20190908_115507_503798_0':  'https://www.youtube.com/watch?v=HKKh-4WHSsg&t=7s',
    'trial_T20190907_011351_027508_0':  'https://www.youtube.com/watch?v=deDf2pnORts',
    'trial_T20190910_055356_172564_0':  'https://www.youtube.com/watch?v=_ys8HY3G94U',
    'trial_T20190919_045956_375063_0':  'https://www.youtube.com/watch?v=9mE_rLrhAhU',
    'trial_T20190918_173742_042769_0':  'https://www.youtube.com/watch?v=7dHM--8H_ws',
    'trial_T20190908_030909_087593_0':  'https://www.youtube.com/watch?v=kXR2SJ4f47w',
    'trial_T20190909_004008_377860_0':  'https://www.youtube.com/watch?v=YBJgvP4Uai8',
    'trial_T20190908_072254_824822_0':  'https://www.youtube.com/watch?v=GZAUr7AToYY',
    'trial_T20190907_181954_161870_0':  'https://www.youtube.com/watch?v=7uDr2uq7gLs',
    'trial_T20190909_113100_297495_0':  'https://www.youtube.com/watch?v=1D8TGusmIaQ',
    'trial_T20190909_150916_682139_0' : 'https://youtu.be/FHGCyxlwp8E',
    'trial_T20190908_192636_561572_0':  'https://youtu.be/KYG5NEfVbjM', 
    'trial_T20190907_013001_399963_0' : 'https://youtu.be/ezmpJhbhCpI',
    'trial_T20190907_074712_870488_0':  'https://youtu.be/civKnURZaR0',
    'trial_T20190908_102840_789300_0': 'https://youtu.be/Dg9dzxXmxB4',
    'trial_T20190909_031040_889814_0': 'https://youtu.be/TpFjTu0nRoA',
    'trial_T20190908_140701_251653_0': 'https://youtu.be/LQpzGDzww1A',
    'trial_T20190909_055649_717880_0': 'https://youtu.be/C9gkb77sF14',
    'trial_T20190907_151802_277016_0': 'https://youtu.be/P9wm8lJwxT4',
    'trial_T20190909_032318_169393_0': 'https://youtu.be/R8fs1wdTMqI',
    'trial_T20190909_123918_983775_0': 'https://youtu.be/1Ha8RLnfsBg',
    'trial_T20190909_123732_489327_0': 'https://youtu.be/4V2hh2cw4No',
    'trial_T20190906_191445_723170_0': 'https://youtu.be/oZ2goybT4pA',
    'trial_T20190907_020329_050110_0': 'https://youtu.be/H-GQkQllSuA',
    'trial_T20190907_170955_824797_0': 'https://youtu.be/8zfLGiywP1A',
    'trial_T20190909_210238_431966_0': 'https://youtu.be/Ti1faQUCSnc',
    'trial_T20190906_224843_443882_0': 'https://youtu.be/z261C5NcQbU',
    'trial_T20190906_214631_761426_0': 'https://youtu.be/6dSWDozBRLU',
    'trial_T20190907_074524_006355_0': 'https://youtu.be/_TobcwcYR9Q',
    'trial_T20190908_114656_768805_0': 'https://youtu.be/AOYa3y_0uCU'
}

In [8]:
root_lookup = {}

for task_id in chosen_train_sanity_task_ids:
    root = mod20_train_sanity_res[task_id]['root']
    root_lookup[task_id] = root
    
for task_id in chosen_valid_seen_task_ids:
    root = mod20_valid_seen_res[task_id]['root']
    root_lookup[task_id] = root
    
for task_id in chosen_valid_unseen_task_ids:
    root = mod20_valid_unseen_res[task_id]['root']
    root_lookup[task_id] = root

## Data Structure

In [9]:
def get_language_annotations(root):
    lang_goals = []
    lang_instrs = []
    path = os.path.join(root, 'pp', 'ann_%d.json' % 0)
    with open(path, 'r') as f:
        ex = json.load(f)
        for i in range(3):
            lang_goals.append(ex['turk_annotations']['anns'][i]['task_desc'])
            lang_instrs.append(ex['turk_annotations']['anns'][i]['high_descs'])
        action_low = [a['discrete_action']['action'] for a in ex['plan']['low_actions']]
        action_high = [a['discrete_action']['action'] for a in ex['plan']['high_pddl']]
    return lang_goals, lang_instrs, action_high, action_low  

def get_ref_instrs(root):
#     flatten_isntr = lambda instr: [word.strip() for sent in instr for word in sent]
    ref_instrs = []
    for i in range(3):
        path = os.path.join(root, 'pp', 'ann_%d.json' % i)
        if os.path.exists(path):
            with open(path, 'r') as f:
                ex = json.load(f)
                ref_instrs.append(ex['ann']['instr'])
    return ref_instrs

In [10]:
def res_by_subgoal_type(task_ids, results):
    res_by_subgoals = defaultdict(lambda: defaultdict(list))
    
    for task_id in task_ids:
        subgoals = results['mod20_p'][task_id]['action_high']
        root = root_lookup[task_id]
        ref_instrs = get_ref_instrs(root)

        for i in range(len(subgoals)-1):
            # extract gold annotation)
            subgoal_name = subgoals[i]
            anns = {'task_id': task_id}
            
            refs = []
            for j, ref in enumerate(ref_instrs):
                ref_instr = [word.strip() for word in ref_instrs[j][i]]
                refs.append(ref_instr)
                anns['gold_ann_{}'.format(j)] = ' '.join(ref_instr)
                
            for k in results.keys():
                p_instr = results[k][task_id]['p_lang_instr'][str(i)]
                anns[k+'_BLEU'] = sentence_bleu(refs, p_instr.split(' '))
                anns[k] = p_instr
            
            res_by_subgoals[subgoal_name][i].append(anns)
            
    return res_by_subgoals

In [11]:
train_sanity_res_by_subgoal_type = res_by_subgoal_type(task_ids=chosen_train_sanity_task_ids, 
                    results= {
                    'mod20_p': mod20_train_sanity_res,
                    'mod21_p': mod21_train_sanity_res,
                    'mod22_p': mod22_train_sanity_res,
                    'mod23_p': mod23_train_sanity_res                        
                    })

valid_seen_res_by_subgoal_type = res_by_subgoal_type(task_ids=chosen_valid_seen_task_ids, 
                    results= {
                    'mod20_p': mod20_valid_seen_res,
                    'mod21_p': mod21_valid_seen_res,
                    'mod22_p': mod22_valid_seen_res,
                    'mod23_p': mod23_valid_seen_res                        
                    })

valid_unseen_res_by_subgoal_type = res_by_subgoal_type(task_ids=chosen_valid_unseen_task_ids, 
                    results= {
                    'mod20_p': mod20_valid_unseen_res,
                    'mod21_p': mod21_valid_unseen_res,
                    'mod22_p': mod22_valid_unseen_res,
                    'mod23_p': mod23_valid_unseen_res                        
                    })

/opt/conda/envs/pytorch-py3.6/lib/python3.6/site-packages/nltk/translate/bleu_score.py:523: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/opt/conda/envs/pytorch-py3.6/lib/python3.6/site-packages/nltk/translate/bleu_score.py:523: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/opt/conda/envs/pytorch-py3.6/lib/python3.6/site-packages/nltk/translate/bleu_score.py:523: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-g

In [12]:
valid_unseen_res_by_subgoal_type.keys()

dict_keys(['GotoLocation', 'PickupObject', 'SliceObject', 'PutObject', 'HeatObject', 'ToggleObject', 'CoolObject'])

In [13]:
train_sanity_res_by_subgoal_type['PickupObject']

defaultdict(list,
            {1: [{'task_id': 'trial_T20190908_115507_503798_0',
               'gold_ann_0': 'pick up the potato from the counter top',
               'gold_ann_1': 'pick up the potato from the counter',
               'gold_ann_2': 'pick up the egg on the counter',
               'mod20_p_BLEU': 0.8408964152537145,
               'mod20_p': 'pick up the potato from the counter .',
               'mod21_p_BLEU': 0.8408964152537145,
               'mod21_p': 'pick up the potato from the counter .',
               'mod22_p_BLEU': 0.5,
               'mod22_p': 'pick up the potato on the counter .',
               'mod23_p_BLEU': 0.31239399369202553,
               'mod23_p': 'pick up a slice of potato from the counter .'},
              {'task_id': 'trial_T20190907_011351_027508_0',
               'gold_ann_0': 'take the towel on the wall of of the towel rack .',
               'gold_ann_1': 'pick up the towel from the rack .',
               'gold_ann_2': 'pick up the 

In [18]:
Subgoal = 'GoToLocation'
valid_unseen_res_by_subgoal_type[Subgoal]

defaultdict(list, {})

In [31]:
split = 'Valid Unseen'
Subgoal = 'GotoLocation'
res = valid_seen_res_by_subgoal_type

ct = 0
for k in sorted(res[Subgoal].keys()):
    
    print ('--------------------------------------------------------------------------')
    print ('{} | Subgoal Index {}'.format(split, k))
    print()
    for task in res[Subgoal][k]:
        ct += 1
        print('TASK: {}'.format(task['task_id']))
        print(urls[task['task_id']])
        print('Gold 0: {}'.format(task['gold_ann_0']))
        print('Gold 1: {}'.format(task['gold_ann_1']))
        print('Gold 2: {}'.format(task['gold_ann_2']))
        print('----------------------')
        print('Model 2.0 BLEU={}'.format(task['mod20_p_BLEU']))
        print(task['mod20_p'])
        print('Model 2.1 BLEU={}'.format(task['mod21_p_BLEU']))
        print(task['mod21_p'])
        print('Model 2.2 BLEU={}'.format(task['mod22_p_BLEU']))
        print(task['mod22_p'])
        print('Model 2.3 BLEU={}'.format(task['mod23_p_BLEU']))
        print(task['mod23_p'])
        print()

--------------------------------------------------------------------------
Valid Unseen | Subgoal Index 0

TASK: trial_T20190909_150916_682139_0
https://youtu.be/FHGCyxlwp8E
Gold 0: move to face the head board of the bed where the purple pillow is
Gold 1: turn around and walk to the middle of the room , then turn right and walk to the window , then turn left to face the bed .
Gold 2: turn left and go to the desk then turn left and go as far as the bed then turn right and go alongside the bed to the nightstand then turn left to face the nightstand
----------------------
Model 2.0 BLEU=0.4971247413297251
turn around and walk to the side of the bed .
Model 2.1 BLEU=0.4971247413297251
turn around and walk to the side of the bed .
Model 2.2 BLEU=0.39721134088567395
turn around and walk to the bed .
Model 2.3 BLEU=0.38219494814404387
turn around and walk to the desk .

TASK: trial_T20190908_192636_561572_0
https://youtu.be/KYG5NEfVbjM
Gold 0: turn around and walk to the table with the kitche

In [32]:
ct

34

In [22]:
32 + 36 + 34

defaultdict(list, {})

In [24]:
valid_unseen_res_by_subgoal_type['GoToLocation'][0]

[]

In [26]:
valid_unseen_res_by_subgoal_type

defaultdict(<function __main__.res_by_subgoal_type.<locals>.<lambda>()>,
            {'GotoLocation': defaultdict(list,
                         {0: [{'task_id': 'trial_T20190909_123918_983775_0',
                            'gold_ann_0': 'move around to the table',
                            'gold_ann_1': 'turn around and cross the room and turn around to face the end of the center table .',
                            'gold_ann_2': 'turn around , go across the room and turn right to face the end of the counter .',
                            'mod20_p_BLEU': 0.1227616815524213,
                            'mod20_p': 'turn around and walk to the end of the room , then turn right and walk to the end of the room , then turn right and walk up to the end of the kitchen island .',
                            'mod21_p_BLEU': 0.6238986072117501,
                            'mod21_p': 'turn around and walk to the end of the counter .',
                            'mod22_p_BLEU': 5.87583260478